In [1]:
#Creation of a sqlite database for the project
#Library
import sqlite3
#Creation of the database
conn = sqlite3.connect('anime_list.db')
#Creation of the cursor (object that will execute the SQL commands)
c = conn.cursor()

# Create tables for foreign keys (type_tv and genre)

c.execute('''
    CREATE TABLE IF NOT EXISTS type_tv (
        id INTEGER PRIMARY KEY,
        type_name TEXT
    )
''')

c.execute('''
    CREATE TABLE IF NOT EXISTS genre (
        id INTEGER PRIMARY KEY,
        genre_name TEXT
    )
''')

#Creation of the table anime

c.execute('''
    CREATE TABLE IF NOT EXISTS anime (
        id INTEGER PRIMARY KEY,
        title TEXT,
        popularity INTEGER,
        last_episode_number INTEGER,
        normalized_rating REAL,
        type_tv_id INTEGER,
        genre_1_id INTEGER,
        genre_2_id INTEGER,
        genre_3_id INTEGER,
        FOREIGN KEY (type_tv_id) REFERENCES type_tv(id),
        FOREIGN KEY (genre_1_id) REFERENCES genre(id),
        FOREIGN KEY (genre_2_id) REFERENCES genre(id),
        FOREIGN KEY (genre_3_id) REFERENCES genre(id)
    )
''')



In [2]:
conn.commit()
conn.close()

In [3]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('anime_list.db')
c = conn.cursor()

df = pd.read_csv('anime_data_cleaned.csv')

try:
    # Insert data into type_tv and genre tables
    c.executemany('INSERT INTO type_tv (type_name) VALUES (?)', [(i,) for i in df['type_tv'].unique()])

    # Insert 'Unknown' genre if it doesn't already exist
    c.execute("INSERT INTO genre (genre_name) SELECT 'Unknown' WHERE NOT EXISTS(SELECT 1 FROM genre WHERE genre_name = 'Unknown')")

    # Insert genres from dataframe
    c.executemany('INSERT INTO genre (genre_name) VALUES (?)', [(i,) for i in pd.concat([df['genre_1'], df['genre_2'], df['genre_3']]).unique()])

    # Get the ids for each type_tv and genre
    type_tv_ids = pd.read_sql_query('SELECT * FROM type_tv', conn).set_index('type_name')['id'].to_dict()
    genre_ids = pd.read_sql_query('SELECT * FROM genre', conn).set_index('genre_name')['id'].to_dict()

    # Replace the type_tv and genre values in the dataframe with the corresponding ids
    df['type_tv_id'] = df['type_tv'].map(type_tv_ids)
    df['genre_1_id'] = df['genre_1'].map(genre_ids)
    df['genre_2_id'] = df['genre_2'].map(genre_ids).fillna(genre_ids['Unknown'])  # replace missing values with the id for 'Unknown'
    df['genre_3_id'] = df['genre_3'].map(genre_ids).fillna(genre_ids['Unknown'])  # replace missing values with the id for 'Unknown'

    # Insert data into anime table
    for i, row in df.iterrows():
        c.execute('''
            INSERT INTO anime (title, popularity, last_episode_number, normalized_rating, type_tv_id, genre_1_id, genre_2_id, genre_3_id)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (row['title'], row['popularity'], row['last_episode_number'], row['normalized_rating'], row['type_tv_id'], row['genre_1_id'], row['genre_2_id'], row['genre_3_id']))
        
        if i % 500 == 0:
            print(f'Inserted {i} rows')

    conn.commit()
    print('Data insertion completed successfully')

except Exception as e:
    print('An error occurred:', e)

finally:
    conn.close()


Inserted 0 rows
Inserted 500 rows
Inserted 1000 rows
Inserted 1500 rows
Inserted 2000 rows
Inserted 2500 rows
Inserted 3000 rows
Inserted 3500 rows
Data insertion completed successfully
